# Lesson  5 Serving Your Personal Tutor with a RESTful API Using FastAPI

Certainly! Here’s your content **converted into structured, readable Markdown** with code blocks and clear headings:

---

# Serving Your Personal Tutor with a RESTful API Using FastAPI

Welcome to the next step in our journey of building a personal tutor service with FastAPI. In the previous lesson, we focused on the `TutorController`, which manages tutoring sessions and handles student queries by interacting with both the model and service layers. Now, we will take a significant step forward by creating a RESTful API for our personal tutor service using **FastAPI**. We'll start by setting up the main FastAPI application, then adapt the `TutorController` to integrate with FastAPI's session management.

---

## Understanding RESTful APIs and FastAPI

RESTful APIs allow different software systems to communicate over the internet using a set of rules for exchanging data. **FastAPI** is a modern, fast (high-performance) web framework for building APIs with Python 3.7+, leveraging Python type hints and providing automatic, interactive API documentation.

### Install FastAPI and Uvicorn

To get started, install FastAPI and Uvicorn with pip:

```bash
pip install fastapi
pip install uvicorn
```

---

## Initializing a FastAPI App

Start by creating your FastAPI app:

```python
from fastapi import FastAPI

# Initialize FastAPI app
app = FastAPI(title="Personal Tutor API")
```

---

## Adding Session Management with Starlette Middleware

FastAPI is built on Starlette, which includes session middleware:

```python
from starlette.middleware.sessions import SessionMiddleware

# Add session middleware
app.add_middleware(
    SessionMiddleware,
    secret_key="your_secret_key_here"
)
```

The `SessionMiddleware` uses cookies to securely manage session data.

---

## Setting Up Static Files and Templates

Serve static files and render HTML templates for a user interface:

```python
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates

# Mount static files directory
app.mount("/static", StaticFiles(directory="static"), name="static")

# Setup Jinja2 templates
templates = Jinja2Templates(directory="templates")
```

---

## Creating the Controller Instance

Instantiate your `TutorController` for handling tutoring operations:

```python
from controllers.tutor_controller import TutorController

# Create an instance of TutorController
tutor_controller = TutorController()
```

---

## Defining Request Models with Pydantic

Use Pydantic to define and validate incoming API requests:

```python
from pydantic import BaseModel

class QueryRequest(BaseModel):
    session_id: str
    query: str
```

---

## Managing Student Sessions

Ensure every student has a unique identifier:

```python
import uuid
from fastapi import Depends, Request

async def get_student_id(request: Request):
    if "student_id" not in request.session:
        request.session["student_id"] = str(uuid.uuid4())
    return request.session["student_id"]
```

---

## Creating the Index Route

Render your homepage with a student session:

```python
@app.get("/")
async def index(request: Request, student_id: str = Depends(get_student_id)):
    return templates.TemplateResponse(
        "tutor.html", 
        {"request": request}
    )
```

---

## Creating New Tutoring Sessions

Define a route to create a new tutoring session:

```python
from fastapi import HTTPException

@app.post("/api/create_session")
async def create_session(student_id: str = Depends(get_student_id)):
    response = tutor_controller.create_session(student_id)
    
    if isinstance(response, tuple):
        error_data = response[0]["error"]
        raise HTTPException(
            status_code=error_data["code"],
            detail=error_data["message"]
        )
    
    return response["data"]
```

---

## Handling Student Queries

Define a route for students to send queries:

```python
@app.post("/api/send_query")
async def send_query(
    query_request: QueryRequest,
    student_id: str = Depends(get_student_id)
):
    response = tutor_controller.send_query(
        student_id,
        query_request.session_id,
        query_request.query
    )
    
    if isinstance(response, tuple):
        error_data = response[0]["error"]
        raise HTTPException(
            status_code=error_data["code"],
            detail=error_data["message"]
        )
    
    return response["data"]
```

---

## Custom Exception Handling

Provide consistent error responses using a custom handler:

```python
from fastapi.responses import JSONResponse

@app.exception_handler(HTTPException)
async def http_exception_handler(request: Request, exc: HTTPException):
    return JSONResponse(
        status_code=exc.status_code,
        content={
            "status": "error",
            "error": {
                "message": exc.detail,
                "code": exc.status_code
            }
        }
    )
```

---

## Running the FastAPI Server

Use Uvicorn to run your application:

```python
import uvicorn

if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )
```

* **host="0.0.0.0"**: Accepts connections from any network interface.
* **port=3000**: Runs on port 3000.
* **reload=True**: Enables hot-reloading during development.

---

## Adapting the TutorController

Your `TutorController` integrates smoothly with FastAPI session management and request handling.

```python
class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()
    
    def create_session(self, student_id):
        if not student_id:
            return self._error_response('Session expired', 401)
        
        session_id = self.tutor_service.create_session(student_id)
        return self._success_response({
            'session_id': session_id,
            'message': 'Tutoring session created successfully'
        })
    
    def send_query(self, student_id, session_id, student_query):
        if not student_id:
            return self._error_response('Session expired', 401)
        
        if not session_id or not student_query:
            return self._error_response('Missing session_id or query', 400)
            
        try:
            tutor_response = self.tutor_service.process_query(
                student_id, 
                session_id, 
                student_query
            )
            return self._success_response({'message': tutor_response})
        except ValueError as e:
            return self._error_response(str(e), 404)
        except RuntimeError as e:
            return self._error_response(str(e), 500)
    
    def _success_response(self, data):
        response = {
            'status': 'success',
            'data': data
        }
        return response
    
    def _error_response(self, message, status_code):
        response = {
            'status': 'error',
            'error': {
                'message': message,
                'code': status_code
            }
        }
        return response, status_code
```

---

## How Students Interact with the Personal Tutor API

1. **Access the Index Route (`/`)**
   Establishes a student session and loads the tutor interface.

2. **Create a Tutoring Session (`/api/create_session`)**
   Returns a unique session ID for the student's tutoring session.

3. **Send a Query (`/api/send_query`)**
   Student sends academic questions; receives AI-generated tutor responses.

---

## Summary and Next Steps

In this lesson, we built a RESTful API for our personal tutor service using FastAPI.
We set up the FastAPI app, defined routes for tutoring operations, and connected everything using session management and controller integration.

As you move on to the practice exercises, experiment with the FastAPI API to reinforce these concepts.
This hands-on work will prepare you for deeper development and additional FastAPI and RESTful API features.

**Keep up the great work, and I look forward to seeing your progress!**


## Serving Your Personal Tutor with a RESTful API Using FastAPI

In this exercise, you'll apply your knowledge of FastAPI to create a simple RESTful API. Your task is to:

Initialize a FastAPI app and reference it as "main:app".
Define an index route at "/" that returns a welcome message.
Set up the app to run on port 3000 and host '0.0.0.0' with reload=True using "main:app" as the application reference.
This exercise is designed to help you build a basic FastAPI application that simply returns a message. At this stage, we are not involving any other layers of our app. Dive in and see how straightforward it can be!

```python
from fastapi import FastAPI
import uvicorn

# TODO: Initialize the FastAPI application


# TODO: Define a route for the index page at "/"
# The route should return "Welcome to the Personal Tutor Service!"

# TODO: Run the FastAPI application with Uvicorn
# - Use "main:app" as the application reference
# - Set host to '0.0.0.0'
# - Set port to 3000
# - Enable reload with reload=True

```

Here’s the complete code for your **basic FastAPI RESTful API exercise**:

```python
from fastapi import FastAPI
import uvicorn

# Initialize the FastAPI application
app = FastAPI()

# Define a route for the index page at "/"
@app.get("/")
async def index():
    return {"message": "Welcome to the Personal Tutor Service!"}

# Run the FastAPI application with Uvicorn
if __name__ == "__main__":
    uvicorn.run(
        "main:app",   # "main" refers to the filename, "app" is the FastAPI instance
        host="0.0.0.0",
        port=3000,
        reload=True
    )
```

**How to run:**

* Save this code in a file named `main.py`.
* Run it using:

  ```bash
  python main.py
  ```
* Open [http://localhost:3000](http://localhost:3000) and you should see your welcome message!

Let me know if you want to add more routes or functionality!


## Building a Simple FastAPI Welcome API

In this task, you will enhance the functionality of a basic FastAPI application by implementing session management within the TutorController. This task will guide you through integrating FastAPI's session capabilities to ensure efficient user session handling. Follow these steps to complete the task:

Update main.py:

Import the TutorController to manage tutoring operations.
Add session handling to your app using SessionMiddleware with a secret key (this lets FastAPI remember who's using your personal tutor).
Create an instance of TutorController to handle tutoring operations.
Ensure a user session is established by calling the ensure_user_session method from the tutor_controller.
Modify /tutor_controller.py:

Remove the test_session dictionary used for simulating sessions.
Update the ensure_user_session method to accept a session parameter and store the user_id in FastAPI's session.
Update the create_session and send_query methods to accept a session parameter and retrieve the user_id from FastAPI's session.
This task will deepen your understanding of integrating FastAPI's session management into a RESTful API. Let's get started!

```python
import uuid
from services.tutor_service import TutorService


class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()
        self.test_session = {}  # TODO: Remove the dictionary used to simulate sessions
    
    # TODO: Update the method signature to accept a session parameter
    def ensure_user_session(self):
        """Ensure user has a session ID in the test session."""
        # TODO: Check if 'user_id' is in FastAPI's session
        if 'user_id' not in self.test_session:
            # TODO: Store new UUID in FastAPI's session
            self.test_session['user_id'] = str(uuid.uuid4())
        # TODO: Return the user_id from FastAPI's session
        return self.test_session['user_id']
    
    # TODO: Update the method signature to accept a session parameter for tutoring session creation
    def create_session(self):
        """Handle tutoring session creation request."""
        # TODO: Retrieve user_id from FastAPI's session
        user_id = self.test_session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        
        chat_id = self.tutor_service.create_session(user_id)
        return {
            'session_id': chat_id,
            'message': 'Tutoring session created successfully'
        }
    
    # TODO: Update the method signature to accept a session parameter
    def send_query(self, session_id, user_query):
        """Handle query sending request."""
        # TODO: Retrieve user_id from FastAPI's session
        user_id = self.test_session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        
        if not session_id or not user_query:
            return {'error': 'Missing session_id or query'}, 400
            
        try:
            ai_response = self.tutor_service.process_query(user_id, session_id, user_query)
            return {'message': ai_response}
        except ValueError as e:
            return {'error': str(e)}, 404
        except RuntimeError as e:
            return {'error': str(e)}, 500


from fastapi import FastAPI, Request
from starlette.middleware.sessions import SessionMiddleware
import uvicorn
# TODO: Import the TutorController


# Initialize FastAPI app
app = FastAPI()

# TODO: Add session middleware with a secret key
# - Use SessionMiddleware from starlette.middleware.sessions
# - Set a secret key to secure user session data

# TODO: Create controller instance


# Define a route for the index page
@app.get("/")
async def index(request: Request):
    # TODO: Call the TutorController's ensure_user_session method
    # Pass request.session to the method to create or retrieve a user ID

    return "Welcome to the Personal Tutor Service!"

# Run the server
if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )

```

Sure! Here’s a **step-by-step implementation** based on your requirements for session management with FastAPI and an updated `TutorController`.

---

## **1. Update `tutor_controller.py`**

**A. Remove `test_session` dictionary**
**B. Update all methods to use FastAPI's `session` (dict-like, from `request.session`)**

```python
import uuid
from services.tutor_service import TutorService

class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()

    def ensure_user_session(self, session):
        """Ensure user has a session ID in FastAPI's session."""
        if 'user_id' not in session:
            session['user_id'] = str(uuid.uuid4())
        return session['user_id']

    def create_session(self, session):
        """Handle tutoring session creation request."""
        user_id = session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        chat_id = self.tutor_service.create_session(user_id)
        return {
            'session_id': chat_id,
            'message': 'Tutoring session created successfully'
        }

    def send_query(self, session, session_id, user_query):
        """Handle query sending request."""
        user_id = session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        if not session_id or not user_query:
            return {'error': 'Missing session_id or query'}, 400
        try:
            ai_response = self.tutor_service.process_query(user_id, session_id, user_query)
            return {'message': ai_response}
        except ValueError as e:
            return {'error': str(e)}, 404
        except RuntimeError as e:
            return {'error': str(e)}, 500
```

---

## **2. Update `main.py`**

**A. Import TutorController and SessionMiddleware**
**B. Add session middleware with a secret key**
**C. Use controller methods with `request.session`**

```python
from fastapi import FastAPI, Request
from starlette.middleware.sessions import SessionMiddleware
from tutor_controller import TutorController  # import your controller!
import uvicorn

app = FastAPI()

# Add session middleware (set your own strong secret key!)
app.add_middleware(SessionMiddleware, secret_key="my_very_secret_key_123")

# Create controller instance
tutor_controller = TutorController()

@app.get("/")
async def index(request: Request):
    # Call ensure_user_session to create/retrieve user_id
    tutor_controller.ensure_user_session(request.session)
    return "Welcome to the Personal Tutor Service!"

# Example endpoints for completeness
@app.post("/session")
async def create_session(request: Request):
    return tutor_controller.create_session(request.session)

@app.post("/query")
async def send_query(request: Request):
    data = await request.json()
    session_id = data.get('session_id')
    user_query = data.get('user_query')
    return tutor_controller.send_query(request.session, session_id, user_query)

if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )
```

---

## **Summary of What Changed**

* Session data is now managed using FastAPI's built-in session (via `request.session`).
* No more global/dictionary hacks; everything persists per user browser/session.
* All controller methods expect a session parameter (the dict-like `request.session`).
* Session is secured with a secret key via `SessionMiddleware`.

---

**This code is ready to use. Just plug it in and you'll have proper user session handling in your TutorController!** If you need a more complete FastAPI route setup or testing tips, let me know!


## Implementing Session Management in FastAPI TutorController

In this task, you'll take a significant step by creating a route that handles tutoring session creation using FastAPI.

Your goal is to add a new endpoint to your FastAPI application that creates a tutoring session when a client makes a request. Specifically, you need to:

Create a route that listens for POST requests at the /api/create_session endpoint
Use the existing tutor_controller to handle the session creation logic by calling its create_session() method and passing the session from the request
Return the response from the controller back to the client
This route is essential for your tutoring service because it allows users to start new learning sessions. Before users can exchange messages with your tutor, they first need to create a tutoring session, which is exactly what this endpoint will enable.

Your FastAPI app is already set up with an index route that welcomes users and ensures they have a session. Now it's time to build the functionality that lets them create tutoring sessions!

```python
from fastapi import FastAPI, Request
from starlette.middleware.sessions import SessionMiddleware
from controllers.tutor_controller import TutorController
import uvicorn

# Initialize FastAPI app
app = FastAPI()

# Add session middleware
app.add_middleware(
    SessionMiddleware,
    secret_key="your_secret_key_here"
)

# Create controller instance
tutor_controller = TutorController()


# Define a route for the index page that ensures a user session
@app.get("/")
async def index(request: Request):
    # Ensure user has a session
    tutor_controller.ensure_student_session(request.session)
    return "Welcome to the Personal Tutor Service!"


# TODO: Define a route for creating a new tutoring session
# - Listen for POST requests at '/api/create_session'
# - Delegate session creation to TutorController's create_session passing the session
# - Return the response from the controller

# Run the server
if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )



```


Absolutely! Here’s how you can implement the `/api/create_session` POST endpoint in your FastAPI app, **delegating logic to your `tutor_controller`** and returning the response to the client.

### **Add This to Your `main.py`**

```python
from fastapi import FastAPI, Request
from starlette.middleware.sessions import SessionMiddleware
from controllers.tutor_controller import TutorController
from fastapi.responses import JSONResponse  # Add this for consistent API response
import uvicorn

app = FastAPI()

app.add_middleware(
    SessionMiddleware,
    secret_key="your_secret_key_here"
)

tutor_controller = TutorController()

@app.get("/")
async def index(request: Request):
    tutor_controller.ensure_student_session(request.session)
    return "Welcome to the Personal Tutor Service!"

# --- YOUR TASK: Implement the /api/create_session endpoint ---
@app.post("/api/create_session")
async def create_tutoring_session(request: Request):
    # Call the controller to handle session creation, passing the session dict
    result = tutor_controller.create_session(request.session)
    # Return as JSON response (if result is tuple, handle error code)
    if isinstance(result, tuple):
        # e.g., ({'error': 'Session expired'}, 401)
        return JSONResponse(content=result[0], status_code=result[1])
    return JSONResponse(content=result)
# --- END OF TASK ---

if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )
```

---

### **Key Points**

* This POST route receives the current user’s session.
* It passes the session to `tutor_controller.create_session`.
* If the controller returns an error (tuple with dict + code), it responds with the right HTTP status.
* If success, it returns a JSON response with the new session data.

**That’s all you need for a robust `/api/create_session` endpoint!**
Ready for next steps (e.g., querying or messaging the tutor)? Just ask!


## Implementing a Tutoring Session Creation Route in FastAPI

Great job on setting up the basic structure of your FastAPI application! Now, let's take an exciting step forward by enabling the ability to send queries to our tutor via the API. This will allow us to interact with our tutoring service in real-time.

Here's what you'll be working on:

Create a new POST route at the /api/send_query endpoint to handle query-sending requests.
Parse the JSON request body to extract necessary data before calling the TutorController.
Pass the extracted request data to the TutorController to manage the logic for processing queries.
Modify the TutorController to extract session_id and user_query from the request body using FastAPI's Request object.
This task will deepen your understanding of building RESTful APIs with FastAPI and bring your tutoring service to life. Dive in and see how these components work together to create a smooth user experience!

```python
import uuid
from services.tutor_service import TutorService


class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()
    
    def ensure_student_session(self, session):
        """Ensure user has a session ID."""
        if 'user_id' not in session:
            session['user_id'] = str(uuid.uuid4())
        return session['user_id']
    
    def create_session(self, session):
        """Handle chat creation request."""
        user_id = session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        
        session_id = self.tutor_service.create_session(user_id)
        return {
            'session_id': session_id,
            'message': 'Chat created successfully'
        }
    
    def send_message(self, session_id, user_message, session):
        """Handle message sending request."""
        user_id = session.get('user_id')
        if not user_id:
            return {'error': 'Session expired'}, 401
        
        if not session_id or not user_message:
            return {'error': 'Missing session_id or message'}, 400
            
        try:
            tutor_response = self.tutor_service.process_query(user_id, session_id, user_message)
            return {'message': tutor_response}
        except ValueError as e:
            return {'error': str(e)}, 404
        except RuntimeError as e:
            return {'error': str(e)}, 500

```


Great! Here’s **how to implement the `/api/send_query` route** and update the controller logic to work perfectly with FastAPI’s request structure and sessions.

---

### 1. **Update Your `TutorController` Method**

You’ll want your `send_message` method to **accept parameters parsed from the JSON body** and also access the session for user identification. The code you posted already does this!

---

### 2. **Create the POST `/api/send_query` Route in `main.py`**

This route will:

* Parse the JSON body to extract `session_id` and `user_message`
* Pass these, along with the session, to `tutor_controller.send_message`
* Return the result

```python
from fastapi import FastAPI, Request
from starlette.middleware.sessions import SessionMiddleware
from controllers.tutor_controller import TutorController
from fastapi.responses import JSONResponse
import uvicorn

app = FastAPI()
app.add_middleware(SessionMiddleware, secret_key="your_secret_key_here")

tutor_controller = TutorController()

@app.get("/")
async def index(request: Request):
    tutor_controller.ensure_student_session(request.session)
    return "Welcome to the Personal Tutor Service!"

@app.post("/api/create_session")
async def create_tutoring_session(request: Request):
    result = tutor_controller.create_session(request.session)
    if isinstance(result, tuple):
        return JSONResponse(content=result[0], status_code=result[1])
    return JSONResponse(content=result)

# --- New route for sending a query/message ---
@app.post("/api/send_query")
async def send_query(request: Request):
    data = await request.json()
    session_id = data.get("session_id")
    user_message = data.get("user_message")
    result = tutor_controller.send_message(session_id, user_message, request.session)
    if isinstance(result, tuple):
        return JSONResponse(content=result[0], status_code=result[1])
    return JSONResponse(content=result)
# --------------------------------------------

if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )
```

---

### **How It Works**

* The client POSTs a JSON payload: `{"session_id": "...", "user_message": "..."}`
* The endpoint pulls these from the body, passes them to the controller alongside the session
* Returns the tutor’s response or an error as JSON

---

**You now have a complete interactive tutoring API with FastAPI, including session management and messaging!**
Ready for more features? Let me know!
